- base: https://www.kaggle.com/code/vadimkamaev/postprocessin-ensemble
- select feats by importances and 3 ensembles (no postprocess) 
- update common class again
- refactoring common classes *

# common class

In [1]:
import sys
import optuna
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold, KFold

from datetime import datetime
from sklearn.decomposition import PCA

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class ObjectiveManager():
    """Class for HyperParameter tuning by Optuna."""
    def __init__(self, model_name, model, config, tr_idx, val_idx):
        self.model_name = model_name
        self.config = config
        self.model = model
        self.tr_idx = tr_idx
        self.val_idx = val_idx
    
    def __call__(self, trial):
        if self.model_name == "lgb":
            # https://lightgbm.readthedocs.io/en/latest/Parameters.html
            check_params = {
                'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
                'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
                'lambda_l2': trial.suggest_float('lambda_l2', 0, 0.1),
                'lambda_l1': trial.suggest_float('lambda_l1', 0, 0.1),
            }
            if self.config["optuna_mode"] == "split": 
                evaluate_func = self.model.lgb_train_and_valid
                
        elif self.model_name == "xgb":
            # https://xgboost.readthedocs.io/en/stable/parameter.html
            check_params = {
                'gamma': trial.suggest_float('gamma', 0, 1),
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
                'subsample': trial.suggest_float('subsample', 0, 1),
                'lambda': trial.suggest_float('lambda', 0, 1),
                'alpha': trial.suggest_float('alpha', 0, 1),
            }
            if self.config["optuna_mode"] == "split": 
                evaluate_func = self.model.xgb_train_and_valid

        elif self.model_name == "cb":
            # https://catboost.ai/en/docs/references/training-parameters/
            check_params = {
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                #'num_leaves': trial.suggest_int('num_leaves', 10, 100),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
                'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.5),
                'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
                'colsample_bylevel': trial.suggest_float('reg_lambda', 0, 1.0),
            }
            if self.config["optuna_mode"] == "split": 
                evaluate_func = self.model.cb_train_and_valid
                
        feats = self.config[self.model_name]["feats"]
        params = self.config[self.model_name]["params"].copy()
        params.update(check_params)
        if self.config["optuna_mode"] == "cv":
            _, val_output, _ = self.model.execute_cv(self.model_name)
            score = self.config["metric_function"](self.config["target"], val_output)
        else:
            _, val_output, _ = evaluate_func(self.tr_idx, self.val_idx, feats, None)
            score = self.config["metric_function"](self.config["target"].iloc[val_idx], val_output)
        return score
    
def param_tuning(config, model_name, model, tr_idx = None, val_idx = None, option = "minimize"):
            
    objective = ObjectiveManager(model_name, model, config, tr_idx, val_idx)
            
    study = optuna.create_study(direction=option) 
    study.optimize(objective, n_trials=config["optuna_trial_num"])
    trial = study.best_trial
    print('Value: ', trial.value)
    return trial.params

In [3]:
class TreeModelling:
    def __init__(self, config):
        self.config = config 
        
    def make_split(self, tr_idx, val_idx, feats):
        x_tr = self.config["train_data"][feats].iloc[tr_idx]
        x_val = self.config["train_data"][feats].iloc[val_idx]
        y_tr = self.config["target"].iloc[tr_idx]
        y_val = self.config["target"].iloc[val_idx]
        return x_tr, x_val, y_tr, y_val
    
        
    def lgb_train_and_valid(self, tr_idx, val_idx, split, verbose=True):
        
        feats = self.config["lgb"]["feats"]
        x_tr, x_val, y_tr, y_val = self.make_split(tr_idx, val_idx, feats)
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"])
        
        model = lgb.train(self.config["lgb"]["params"], lgb.Dataset(x_tr, y_tr), 
                          self.config["lgb"]["num_round"], valid_sets=[lgb.Dataset(x_val, y_val)], 
                          callbacks = [lgb.early_stopping(stopping_rounds=self.config["lgb"]["es_round"], verbose=verbose), 
                                       lgb.log_evaluation(self.config["lgb"]["verbose_eval"])]) 
        feat_imp_df["imp"+str(split)] = model.feature_importance()
        valid_pred = model.predict(x_val)
    
        return model, valid_pred, feat_imp_df
    
    def lgb_test(self, models, test):
        feats = self.config["lgb"]["feats"]
        test_pred = [model.predict(test[feats]) for model in models]
        test_pred = np.mean(test_pred, axis=0)
        return test_pred
    

    def xgb_train_and_valid(self, tr_idx, val_idx, split):
        
        feats = self.config["xgb"]["feats"]
        x_tr, x_val, y_tr, y_val = self.make_split(tr_idx, val_idx, feats)
        
        xgb_eval = xgb.DMatrix(x_val, label = y_val)
        model = xgb.train(self.config["xgb"]["params"], xgb.DMatrix(x_tr, label = y_tr), 
                          self.config["xgb"]["num_round"], evals = [(xgb_eval, "eval")], 
                          early_stopping_rounds=self.config["xgb"]["es_round"], 
                          verbose_eval = self.config["xgb"]["verbose_eval"])
        feat_imp_df = pd.DataFrame(model.get_score(importance_type="total_gain").items(), 
                                   columns =["feat", "imp"+str(split)])
        valid_pred = model.predict(xgb_eval, iteration_range=(0, model.best_ntree_limit))
    
        return model, valid_pred, feat_imp_df
    
    def xgb_test(self, models, test):
        feats = self.config["xgb"]["feats"]
        dtest = xgb.DMatrix(test[feats])
        test_pred = [model.predict(dtest, iteration_range=(0, model.best_ntree_limit)) for model in models]
        test_pred = np.mean(test_pred, axis=0)
        return test_pred

    
    def cb_train_and_valid(self, tr_idx, val_idx, split):
        
        feats = self.config["cb"]["feats"]
        x_tr, x_val, y_tr, y_val = self.make_split(tr_idx, val_idx, feats)
        feat_imp_df = pd.DataFrame(feats, columns = ["feat"]) 
        
        if self.config["task_type"] == "classification":
            model = CatBoostClassifier(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], 
                      early_stopping_rounds=self.config["cb"]["es_round"],
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict_proba(x_val)[:,1]
        elif self.config["task_type"]:
            model = CatBoostRegressor(**self.config["cb"]["params"])
            model.fit(x_tr, y_tr, eval_set=[(x_val, y_val)], 
                      early_stopping_rounds=self.config["cb"]["es_round"], 
                      verbose_eval = self.config["cb"]["verbose_eval"])
            valid_pred = model.predict(x_val)
        
        feat_imp_df["imp"+str(split)] = model.get_feature_importance()
    
        return model, valid_pred, feat_imp_df
    
    def cb_test(self, models, test):
        feats = self.config["cb"]["feats"]
        if self.config["task_type"] == "classification":
            test_pred = [model.predict_proba(test[feats])[:,1] for model in models]
        else:
            test_pred = [model.predict(test[feats]) for model in models]
        test_pred = np.mean(test_pred, axis=0)        
        return test_pred
    
    
    def execute_cv(self, model_name):
            
        models = []
        feats = self.config[model_name]["feats"]
        valid_output = np.zeros(len(self.config["target"]))
        
        if model_name == "lgb":
            func = self.lgb_train_and_valid
        elif model_name == "xgb":
            func = self.xgb_train_and_valid
        elif model_name == "cb":
            func = self.cb_train_and_valid
        
        ####
        cv_scheme = MultilabelStratifiedKFold(5, shuffle=True, random_state=42).split(self.config["train_data"],
                                                                                      train_stratify)
        ####
    
        for split, (tr_idx, val_idx) in enumerate(cv_scheme): 
            if split == 0:
                model, valid_pred, feat_imp_df = func(tr_idx, val_idx, split)
            else:
                model, valid_pred, tmp_imp_df = func(tr_idx, val_idx, split)
                feat_imp_df = pd.merge(feat_imp_df, tmp_imp_df, on = "feat", how = "inner")
            valid_output[val_idx] = valid_pred
            models.append(model)
    
        feat_imp_df["sum"] = feat_imp_df.iloc[:,1:].sum(axis=1)
        val_score = self.config["metric_function"](self.config["target"], valid_output)
        print("validation score: ", val_score)
        print(" ")
    
        return models, valid_output, feat_imp_df

# preprocess

In [4]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')


train['EJ'] = train['EJ'].map({'A': 0, 'B': 1})
test['EJ']  = test['EJ'].map({'A': 0, 'B': 1})

# process epsilon
train = pd.merge(train, greeks, on = "Id", how = "inner")
train_stratify = train[["Class", "Beta", "Delta", "Gamma"]] 
train["Epsilon_ordinal"] = train["Epsilon"].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal() if x != "Unknown" else np.nan)

org_features = [n for n in train.columns if n not in ['Class', 'Id', 'Alpha', "Beta", "Gamma", "Delta", "Epsilon"]]
test_times = pd.DataFrame([train.Epsilon_ordinal.max() + 1] * len(test), columns = ["Epsilon_ordinal"])
final_test = pd.concat((test, test_times), axis=1)

# fill missing value
train.fillna(-999, inplace=True)
final_test.fillna(-999, inplace=True)

# add pca columns
pca_feat_num = 15
pca_cols = ["pca"+str(i+1) for i in range(pca_feat_num)]
pca = PCA(n_components=pca_feat_num,random_state=42)
pca_train = pca.fit_transform(train[org_features])
pca_test = pca.transform(final_test[org_features])
pca_train = pd.DataFrame(pca_train, columns=pca_cols)
pca_test = pd.DataFrame(pca_test, columns=pca_cols)
train = pd.concat([train, pca_train],axis=1)
final_test = pd.concat([final_test, pca_test],axis=1)

print(train.shape, final_test.shape)

(617, 79) (5, 73)


# config

In [5]:
from sklearn.metrics import log_loss
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return log_loss(y_true, y_pred, sample_weight = 1/nc[y_true], eps=1e-15)

config = {
    "xgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 50,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'verbosity': 0,
            'seed': 42
        },   
    },
    
    "lgb": {
        "feats": org_features + pca_cols,
        "num_round" : 1000,
        "es_round" : 20,
        "verbose_eval": 100,
        "params" : {
            'objective': 'binary', 
            'metric': 'binary_logloss', 
            'boosting': 'goss',
            'verbose': -1,
            'seed': 42,
            'is_unbalance': True
        },   
    },

    "cb": {
        "feats": org_features + pca_cols,
        "es_round" : 40,
        "verbose_eval": 100,
        "params" : {
            'loss_function': 'Logloss',
            'iterations': 1000,  
            'random_seed': 42,
            'use_best_model' : True
        },
    },
    "task_type": "classification",
    "train_data": train,
    "test_data": final_test,
    "target": train.Class,
    "metric_function" : balanced_log_loss,
    # optuna config
    "optuna_trial_num": 5,
    "optuna_mode" : "cv",
}

# tree model

In [6]:
treemodel = TreeModelling(config)

## lgb

In [7]:
lgb_models, lgb_val_output, lgb_imp_df = treemodel.execute_cv("lgb")
lgb_test_output = treemodel.lgb_test(lgb_models, config["test_data"])

Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
validation score:  0.22179991403414676
 


In [8]:
lgb_best_params = param_tuning(config = config, model_name = "lgb", model = treemodel)
lgb_best_params

[I 2023-11-18 01:05:53,094] A new study created in memory with name: no-name-b5695834-d807-4b7b-a3aa-df34175d5143


Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839


[I 2023-11-18 01:05:54,761] Trial 0 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 71, 'learning_rate': 0.21606492576359745, 'feature_fraction': 0.6608910760693518, 'bagging_fraction': 0.8424804482547122, 'min_child_samples': 10, 'lambda_l2': 0.002318107061900232, 'lambda_l1': 0.08315354314938368}. Best is trial 0 with value: 0.22179991403414676.


validation score:  0.22179991403414676
 
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839


[I 2023-11-18 01:05:56,458] Trial 1 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 76, 'learning_rate': 0.48597269316971803, 'feature_fraction': 0.7343401421922306, 'bagging_fraction': 0.8692376672155666, 'min_child_samples': 5, 'lambda_l2': 0.0775177431766047, 'lambda_l1': 0.01688443631801524}. Best is trial 0 with value: 0.22179991403414676.


...

[I 2023-11-18 01:05:59,939] Trial 3 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 91, 'learning_rate': 0.39045139678936347, 'feature_fraction': 0.6444971986230041, 'bagging_fraction': 0.6627849341152106, 'min_child_samples': 5, 'lambda_l2': 0.003937299850873444, 'lambda_l1': 0.0020529119492702533}. Best is trial 0 with value: 0.22179991403414676.


Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839
validation score:  0.22179991403414676
 
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.0753021
Early stopping, best iteration is:
[133]	valid_0's binary_logloss: 0.0568181
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.190418
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.181247
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.173883
Early stopping, best iteration is:
[117]	valid_0's binary_logloss: 0.171541
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.166974
Early stopping, best iteration is:
[176]	valid_0's binary_logloss: 0.148852
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[71]	valid_0's binary_logloss: 0.156839


[I 2023-11-18 01:06:02,667] Trial 4 finished with value: 0.22179991403414676 and parameters: {'num_leaves': 17, 'learning_rate': 0.3842755399569132, 'feature_fraction': 0.6767781818198513, 'bagging_fraction': 0.6235528373641517, 'min_child_samples': 4, 'lambda_l2': 0.05534887017307383, 'lambda_l1': 0.028463241064023405}. Best is trial 0 with value: 0.22179991403414676.


validation score:  0.22179991403414676
 
Value:  0.22179991403414676


{'num_leaves': 71,
 'learning_rate': 0.21606492576359745,
 'feature_fraction': 0.6608910760693518,
 'bagging_fraction': 0.8424804482547122,
 'min_child_samples': 10,
 'lambda_l2': 0.002318107061900232,
 'lambda_l1': 0.08315354314938368}

## xgb

In [9]:
xgb_models, xgb_val_output, xgb_imp_df = treemodel.execute_cv("xgb")
xgb_test_output = treemodel.xgb_test(xgb_models, config["test_data"])

[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[73]	eval-logloss:0.16337
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594
validation score:  0.31216516835643554
 


In [10]:
xgb_best_params = param_tuning(config = config, model_name = "xgb", model = treemodel)
xgb_best_params

[I 2023-11-18 01:06:05,124] A new study created in memory with name: no-name-e7087a31-2f99-4dcc-ac81-b4cad81f4bd4


[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[144]	eval-logloss:0.15872
[0]	eval-logloss:0.52212
[72]	eval-logloss:0.20594


[I 2023-11-18 01:06:07,470] Trial 0 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.666296797420977, 'max_depth': 7, 'num_leaves': 85, 'learning_rate': 0.36023275970736934, 'min_child_weight': 1, 'max_delta_step': 2, 'subsample': 0.07006128213587715, 'lambda': 0.8111668883534335, 'alpha': 0.05512814437811153}. Best is trial 0 with value: 0.31216516835643554.


validation score:  0.31216516835643554
 
[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[293]	eval-logloss:0.07732
[0]	eval-logloss:0.52655
[78]	eval-logloss:0.15152
[0]	eval-logloss:0.50637
[73]	eval-logloss:0.16337
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[145]	eval-logloss:0.15881
[0]	eval-logloss:0.52212
[73]	eval-logloss:0.20678


[I 2023-11-18 01:06:09,814] Trial 1 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.01671845237795755, 'max_depth': 9, 'num_leaves': 41, 'learning_rate': 0.29768581917864334, 'min_child_weight': 2, 'max_delta_step': 2, 'subsample': 0.04242094836640775, 'lambda': 0.7208517725554118, 'alpha': 0.07281482981385534}. Best is trial 0 with value: 0.31216516835643554.


...

[I 2023-11-18 01:06:14,505] Trial 3 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.9161483212170861, 'max_depth': 2, 'num_leaves': 70, 'learning_rate': 0.02996066332979274, 'min_child_weight': 1, 'max_delta_step': 6, 'subsample': 0.28859938577689126, 'lambda': 0.17412933348010395, 'alpha': 0.9460867129294462}. Best is trial 0 with value: 0.31216516835643554.


validation score:  0.31216516835643554
 
[0]	eval-logloss:0.50062
[100]	eval-logloss:0.07995
[200]	eval-logloss:0.07652
[294]	eval-logloss:0.07731
[0]	eval-logloss:0.52655
[79]	eval-logloss:0.15150
[0]	eval-logloss:0.50637
[74]	eval-logloss:0.16255
[0]	eval-logloss:0.52571
[100]	eval-logloss:0.15744
[144]	eval-logloss:0.15872
[0]	eval-logloss:0.52212
[73]	eval-logloss:0.20678


[I 2023-11-18 01:06:16,823] Trial 4 finished with value: 0.31216516835643554 and parameters: {'gamma': 0.4545729895453162, 'max_depth': 3, 'num_leaves': 100, 'learning_rate': 0.3587800139694318, 'min_child_weight': 6, 'max_delta_step': 8, 'subsample': 0.21708634944623684, 'lambda': 0.18853139172757183, 'alpha': 0.3960629808842593}. Best is trial 0 with value: 0.31216516835643554.


validation score:  0.31216516835643554
 
Value:  0.31216516835643554


{'gamma': 0.666296797420977,
 'max_depth': 7,
 'num_leaves': 85,
 'learning_rate': 0.36023275970736934,
 'min_child_weight': 1,
 'max_delta_step': 2,
 'subsample': 0.07006128213587715,
 'lambda': 0.8111668883534335,
 'alpha': 0.05512814437811153}

In [11]:
cb_models, cb_val_output, cb_imp_df = treemodel.execute_cv("cb")
cb_test_output = treemodel.cb_test(cb_models, config["test_data"])

Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 69.9ms	remaining: 1m 9s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 860ms	remaining: 7.65s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.65s	remaining: 6.57s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.44s	remaining: 5.68s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.29s	remaining: 4.91s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.09s	remaining: 4.08s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.9s	remaining: 3.25s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.7s	remaining: 2.43s
800:	learn: 0.0042617	test: 0.0891180	best: 0.0891180 (800)	total: 6.5s	remaining: 1.61s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.08903141659
bestIteration = 803

Shrink model to first 804 iterations.
Learning rate set to 0.0266

In [12]:
cb_best_params = param_tuning(config = config, model_name = "cb", model = treemodel)
cb_best_params

[I 2023-11-18 01:06:38,133] A new study created in memory with name: no-name-39f40bd1-75a6-4894-a365-3342a1f5e60e


Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 9.07ms	remaining: 9.07s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 812ms	remaining: 7.23s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.62s	remaining: 6.44s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.46s	remaining: 5.72s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.26s	remaining: 4.86s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.05s	remaining: 4.03s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.84s	remaining: 3.21s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.63s	remaining: 2.4s
800:	learn: 0.0042617	test: 0.0891180	best: 0.0891180 (800)	total: 6.42s	remaining: 1.59s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.08903141659
bestIteration = 803

Shrink model to first 804 iterations.
Learning rate set to 0.02

[I 2023-11-18 01:06:58,457] Trial 0 finished with value: 0.3645940857907282 and parameters: {'max_depth': 9, 'reg_lambda': 0.20976982415507472, 'learning_rate': 0.13263393173627286, 'min_child_samples': 1}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.39s	remaining: 5.54s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
validation score:  0.3645940857907282
 
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.97ms	remaining: 8.96s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 807ms	remaining: 7.18s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.64s	remaining: 6.54s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.43s	remaining: 5.65s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.22s	remaining: 4.81s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.01s	remaining: 3.99s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.8s	remaining: 3.19s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.69s	rem

[I 2023-11-18 01:07:19,454] Trial 1 finished with value: 0.3645940857907282 and parameters: {'max_depth': 4, 'reg_lambda': 0.17998462252731118, 'learning_rate': 0.43828532968227335, 'min_child_samples': 8}. Best is trial 0 with value: 0.3645940857907282.


...

[I 2023-11-18 01:08:01,045] Trial 3 finished with value: 0.3645940857907282 and parameters: {'max_depth': 6, 'reg_lambda': 0.8285465902613165, 'learning_rate': 0.46939934324073684, 'min_child_samples': 7}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.49s	remaining: 5.78s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
validation score:  0.3645940857907282
 
Learning rate set to 0.026623
0:	learn: 0.6658456	test: 0.6657945	best: 0.6657945 (0)	total: 8.56ms	remaining: 8.55s
100:	learn: 0.1123480	test: 0.1832610	best: 0.1832610 (100)	total: 815ms	remaining: 7.25s
200:	learn: 0.0492788	test: 0.1381917	best: 0.1381917 (200)	total: 1.61s	remaining: 6.41s
300:	learn: 0.0253975	test: 0.1166001	best: 0.1166001 (300)	total: 2.41s	remaining: 5.61s
400:	learn: 0.0143491	test: 0.1034707	best: 0.1034707 (400)	total: 3.21s	remaining: 4.8s
500:	learn: 0.0094645	test: 0.0988917	best: 0.0988917 (500)	total: 4.01s	remaining: 3.99s
600:	learn: 0.0068158	test: 0.0956670	best: 0.0952825 (577)	total: 4.8s	remaining: 3.19s
700:	learn: 0.0052091	test: 0.0923589	best: 0.0923589 (700)	total: 5.61s	rema

[I 2023-11-18 01:08:21,996] Trial 4 finished with value: 0.3645940857907282 and parameters: {'max_depth': 6, 'reg_lambda': 0.6007171966871011, 'learning_rate': 0.11830260594732224, 'min_child_samples': 6}. Best is trial 0 with value: 0.3645940857907282.


300:	learn: 0.0218487	test: 0.1931048	best: 0.1908997 (267)	total: 2.47s	remaining: 5.74s
Stopped by overfitting detector  (40 iterations wait)

bestTest = 0.1908997058
bestIteration = 267

Shrink model to first 268 iterations.
validation score:  0.3645940857907282
 
Value:  0.3645940857907282


{'max_depth': 9,
 'reg_lambda': 0.20976982415507472,
 'learning_rate': 0.13263393173627286,
 'min_child_samples': 1}